In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
from time import sleep
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
def formatted_today():
    import datetime
    now = datetime.datetime.now()
    today = "{0:%Y-%m-%d}".format(now)
    return today

# 表示を見やすくするための横線を引くだけ
def print_line():
    print("---------------------------------------------------------------------------------------------------------------------------------------")

# サーバーの連続アクセスを避けるために休憩時間(second)
def sleep_(sec):
    print("過剰な連続リクエスト回避のsleep:"+str(sec)+"秒")
    sleep(sec)
    

#     b=パラメータに代入する数字リストを作る。
# b= パラメータには検索結果ページに表示する商品の開始番号がはいる
# n=パラメータには検索結果ページ一ページごとの表示数がはいる
# 例えばb=101でn=100の時、101番目から200番目の商品が表示される
# 引数には検索結果の商品数を与える。
# 例:検索結果が217件でn=100なら
#  get_product_list(total_product=217,n=100)で実行
# →[1,101,201]
def b_list(total_product,n):
    page_count = total_product//n
    page_list = []
    for i in range(0, page_count+1):
        page_list.append(i*100+1)
    return page_list


# 同時に検索する検索ワードを繋げてURL用に加工する
# "airpods", "第2世代", "新品"の３ワードを繋げようと思ったら
# get_combined_params(["airpods", "第2世代","新品"])と、リストを入れる
# →"airpods+第2世代+新品"    

def get_combined_params(params):
    return "+".join(params)


b_list(1100,100)

[1, 101, 201, 301, 401, 501, 601, 701, 801, 901, 1001, 1101]

In [18]:
def get_search_result_page_single(b,products):
    # 今回は終了したオークションに検索を限定する(進行中と分けると実装がめんどいので)
    # (進行中のオークションに対応させたかったら連絡ください)
    search_mode = "closedsearch"
    url_without_param = "https://auctions.yahoo.co.jp/"+search_mode+"/"+search_mode+"?n=100"
    r = requests.get(url_without_param+"&p="+get_combined_params(products)+"&va="+get_combined_params(products)+"&b="+str(b))
    soup = BeautifulSoup(r.text,"html.parser")
    detail_link_tags = soup.select(".Product__titleLink")
    return detail_link_tags


# 商品詳細ページ100ページから１件ずつデータを取得した
# 100行のDataFrameを一つ返す
# 引数
# detail_link_tags: 100個のリンク
# b: 開始番号。1とか101。この名前でcsvを保存する
# query: 検索文字列。リストじゃなくて+で繋がったやつ。 "airpods+第2世代+新品" この名前でフォルダを作る
def create_dataframe(detail_link_tags,b,query):
    # あらかじめ各列のデータを100件ずつ収める予定の空リストを作っておく。
    # これを検索結果一ページごとに空のDataFrameに代入してDataFrameを作るのが最も高速。
    is_free_shipping_list = []
    product_name_list     = []
    start_price_list      = []
    end_price_list        = []
    is_prompt_list        = []
    bidders_list=[]
    start_time_list       = []
    end_time_list         = []
    timedelta_list=[]
    description_list      = []
    good_list=[]
    bad_list=[]
    capture_count_list=[]
    for link_tag in detail_link_tags:
        link_url = link_tag.get("href")
        sleep_(0.5) #まぁ0.5秒くらい開けとけば割と平気
        print(f"{len(is_free_shipping_list) + 1}商品目: {link_url} の取得開始")
        r = requests.get(link_url)
        soup = BeautifulSoup(r.text,"html.parser")
        
#       エラーチェック: リンクは残っているが削除されている商品が途中にある
#       削除済み商品も列数にカウントしておきたいので、全てにNoneを代入し、商品説明だけ「削除済み」の旨を代入する  
#     　削除済み商品はそもそも商品名がない
        if(not soup.select(".highlightWordSearch h1")):
            print("※欠損商品発見,スキップ")
            product_name_list.append(None)
            is_free_shipping_list.append(None)
            start_price_list.append(None)
            end_price_list.append(None)
            is_prompt_list.append(None)
            start_time_list.append(None)
            end_time_list.append(None)
            timedelta_list.append(None)
            description_list.append("削除済み")
            bidders_list.append(None)
            good_list.append(None)
            bad_list.append(None)
            capture_count_list.append(None)
            
            continue
            


        #   送料無料かどうか(送料無料ダミー)
        #  送料無料のボックスを持つページには.decFreeShiping クラスがある あるなら1ないなら0
        if(soup.select(".decFreeShiping")):
            is_free_shipping_list.append(1)
        else:
            is_free_shipping_list.append(0)
            
      #落札人数  
        bidders=soup.select("b[property='auction:Bids']")[0].text.strip()
        bidders=int(bidders)
        bidders_list.append(bidders)



        #  商品名(データフレームの列名は適宜書き換えてね)
        #  商品名は
        product_name = soup.select(".highlightWordSearch h1")[0].text
        product_name_list.append(product_name)

    #     価格については開始価格、即決価格、落札価格全部取ることにする

    # 　開始価格　<td property=auction:StartPrice> に入ってる
        start_price = soup.select("td[property='auction:StartPrice']")[0].text.strip().replace(",","").replace("円", "").replace(" ","")
        start_price = int(start_price)
        start_price_list.append(start_price)

    #   即決価格か現在価格のどちらかが存在するので、どちらも「終了価格:end_price」とする
    # 　即決ダミーを後の分析のために残しておく「即決ダミー: is_prompt」即決なら1、現在価格なら0

    #   即決価格は<p property="auction:BidOrBuyPrice" class="decTxtBuyPrice Price__value">18,290円</p>に入ってる
        if(soup.select("p.decTxtBuyPrice")):
            prompt_price = soup.select("p.decTxtBuyPrice")[0].text.strip().replace(",","").replace("円","")
            prompt_price = int(prompt_price)
            end_price_list.append(prompt_price)
            is_prompt_list.append(1)
    #   現在価格は<p property="auction:Price" class="decTxtAucPrice Price__value">18,500円</p>に入っている
        elif(soup.select("p.decTxtAucPrice")):
            current_price = soup.select("p.decTxtAucPrice")[0].text.strip().replace(",","").replace("円","")
            current_price = int(current_price)
            end_price_list.append(current_price)
            is_prompt_list.append(0)
            

    #   開始時間と終了時間

    # 　開始時間タグは<td property="auction:StartTime">9月 11日(水)22時 53分</td>にあるにはあるが、フォーマットが面倒臭い。
    # 　下の方に<span style="display:none" property="auction:StartTimeUT">1568210002</span>なるタグを発見。「UNIX標準時」で検索。
    #   UNIX標準時からの時刻の作成はめちゃくちゃ簡単なのでこちらを採用。

        start_time = soup.select("span[property='auction:StartTimeUT']")[0].text
        start_time = int(start_time)
        start_time =  datetime.utcfromtimestamp(start_time)
        start_time_list.append(start_time)

    #   終了時間は<span style="display:none" property="auction:EndTimeUT">1568250325</span>に入ってる。
        end_time = soup.select("span[property='auction:EndTimeUT']")[0].text
        end_time = int(end_time)
        end_time = datetime.utcfromtimestamp(end_time)
        end_time_list.append(end_time)
        
        timedelta=end_time-start_time
        timedelta=timedelta.total_seconds()
        timedelta=timedelta//3600
        timedelta_list.append(timedelta)
        
        
        

    #   商品説明。何の情報が欲しいのかわからないのでとりあえずそのまま本文を代入
    #   商品説明は<div id="acMdUsrPrv" class="modUsrPrv highlightWordSearch">に入っている
    # 　spanタグやらに囲まれてごちゃごちゃしているので、get_text()でタグを除去
        description = soup.select("#acMdUsrPrv")[0].get_text().replace("\n"," ").replace("\xa0", " ").strip()
        description_list.append(description)
        
        target = soup.select(".decPwrBox table tr:nth-child(2) td")[0]
        target = str(target)
        p = re.compile(r"<[^>]*?>") #余計な<b>やら<span>やら<img>やらを取り除く 
        target = p.sub("",target).replace("\n", "").replace("\xa0", "") #余計な改行文字や全角空白を取り除く
        pe = re.compile(r"(\d+-\d+)") # (良い評価(数値)-悪い評価(数値)) 例： (921-1) の形式で入っているので、その部分だけ抽出する
        evals = pe.search(target).group().split("-")
        good = evals[0] #良い評価
        bad = evals[1] # 悪い評価
        print("good:", good, "bad:", bad)
        good_list.append(good)
        bad_list.append(bad)
        print("good_list:", len(good_list), "bad_list:", len(bad_list))
        
#   画像枚数
#         capture_count=soup.select(".decBg01 span")[0].get_text().replace("(全","").replace("枚)","").strip()
#　修正： 文字情報ではなく画像の枚数を数えることにする
        capture_count = len(soup.select("#acMdMainPhoto  .untBody .pts01 img")) #文字
        capture_count = int(capture_count) #整数
        print("画像枚数：", capture_count)
        capture_count_list.append(capture_count)

    #   ログ
        print("取得完了")

    print("全リンク取得完了")
    #     DataFrame作成(データフレームの列名は適宜書き換えてね)
    df = pd.DataFrame()
    df["is_free_shipping"] = is_free_shipping_list
    df["product_name"]     = product_name_list
    df["start_price"]      = start_price_list
    df["end_price"]        = end_price_list
    df["is_prompt"]        = is_prompt_list
    df["start_time"]       = start_time_list
    df["end_time"]         = end_time_list
    df["timedelta"]=timedelta_list
    df["bidders"]=bidders_list
    df["setsumei"]=description_list
    print("good_list:", len(good_list), "bad_list:", len(bad_list))
    df["good"]=good_list
    df["bad"]=bad_list
    df["capture_count"]=capture_count_list
    

    
    #   おまけ: 開始日時と終了日時の差の日数
    #   timedelta列：日付差データ全部(日数とか時間とか)
    #   days_diff列: 日付差だけ  (timedelta.days関数で抽出できる)
    

    
#     os.makedirsはフォルダを作る。exist_okをTrueにしないと、既にあるフォルダを作るとエラー
    os.makedirs(query,exist_ok=True)
#     保存して終わり。 "101.csv"
    df.to_csv(query+"/"+query+str(b)+".csv")

In [ ]:
product_b_list = b_list(1195,100)#[1,101,201,...,1001,1101]の、b=パラメータに入れる数値を作成。

# products = ["airpods","第2世代","新品"] #ここに検索ワードを"リストで"入れる
products = ["airpods","第2世代","新品"]

for number in product_b_list:
    print(str(number)+"商品から取得")
    # b=に1や101を代入。productsにワードのリストを代入。
    # 検索結果一ページごとのリンクタグが100個帰ってくるのでtagsに代入 
    tags = get_search_result_page_single(b=number,products=products)
          
    create_dataframe(detail_link_tags=tags, b=number,query=get_combined_params(products))
    print(str(number+99)+"商品まで取得完了")


1商品から取得
過剰な連続リクエスト回避のsleep:0.5秒
1商品目: https://page.auctions.yahoo.co.jp/jp/auction/c772444948 の取得開始
good: 232 bad: 0
good_list: 1 bad_list: 1
画像枚数： 2
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
2商品目: https://page.auctions.yahoo.co.jp/jp/auction/m372149018 の取得開始
good: 176 bad: 2
good_list: 2 bad_list: 2
画像枚数： 2
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
3商品目: https://page.auctions.yahoo.co.jp/jp/auction/p724166999 の取得開始
good: 12 bad: 0
good_list: 3 bad_list: 3
画像枚数： 4
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
4商品目: https://page.auctions.yahoo.co.jp/jp/auction/h434231306 の取得開始
good: 2744 bad: 10
good_list: 4 bad_list: 4
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
5商品目: https://page.auctions.yahoo.co.jp/jp/auction/r360447016 の取得開始
good: 12 bad: 0
good_list: 5 bad_list: 5
画像枚数： 4
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
6商品目: https://page.auctions.yahoo.co.jp/jp/auction/425877060 の取得開始
good: 241 bad: 0
good_list: 6 bad_list: 6
画像枚数： 9
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
7商品目: https://page.auctions.yahoo.co.jp/jp/auction/d398012971 の取得開始
good: 232 bad: 1
go

good: 275 bad: 1
good_list: 55 bad_list: 55
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
56商品目: https://page.auctions.yahoo.co.jp/jp/auction/r359106649 の取得開始
good: 1071 bad: 4
good_list: 56 bad_list: 56
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
57商品目: https://page.auctions.yahoo.co.jp/jp/auction/o353085844 の取得開始
good: 1071 bad: 4
good_list: 57 bad_list: 57
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
58商品目: https://page.auctions.yahoo.co.jp/jp/auction/x658016469 の取得開始
good: 1071 bad: 4
good_list: 58 bad_list: 58
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
59商品目: https://page.auctions.yahoo.co.jp/jp/auction/n380750756 の取得開始
good: 1071 bad: 4
good_list: 59 bad_list: 59
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
60商品目: https://page.auctions.yahoo.co.jp/jp/auction/d399048877 の取得開始
good: 1071 bad: 4
good_list: 60 bad_list: 60
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
61商品目: https://page.auctions.yahoo.co.jp/jp/auction/u312653253 の取得開始
good: 663 bad: 37
good_list: 61 bad_list: 61
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
62商品目: https:

10商品目: https://page.auctions.yahoo.co.jp/jp/auction/t682266344 の取得開始
good: 1071 bad: 4
good_list: 10 bad_list: 10
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
11商品目: https://page.auctions.yahoo.co.jp/jp/auction/b425448163 の取得開始
good: 1071 bad: 4
good_list: 11 bad_list: 11
画像枚数： 2
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
12商品目: https://page.auctions.yahoo.co.jp/jp/auction/d396237390 の取得開始
good: 683 bad: 4
good_list: 12 bad_list: 12
画像枚数： 2
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
13商品目: https://page.auctions.yahoo.co.jp/jp/auction/428283991 の取得開始
good: 309 bad: 4
good_list: 13 bad_list: 13
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
14商品目: https://page.auctions.yahoo.co.jp/jp/auction/h431440060 の取得開始
good: 275 bad: 1
good_list: 14 bad_list: 14
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
15商品目: https://page.auctions.yahoo.co.jp/jp/auction/429957147 の取得開始
good: 115 bad: 0
good_list: 15 bad_list: 15
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
16商品目: https://page.auctions.yahoo.co.jp/jp/auction/c768484813 の取得開始
good: 115 bad: 0
good_list: 16 

65商品目: https://page.auctions.yahoo.co.jp/jp/auction/d396272259 の取得開始
good: 309 bad: 4
good_list: 65 bad_list: 65
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
66商品目: https://page.auctions.yahoo.co.jp/jp/auction/e388605834 の取得開始
good: 1071 bad: 4
good_list: 66 bad_list: 66
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
67商品目: https://page.auctions.yahoo.co.jp/jp/auction/c766712309 の取得開始
good: 1071 bad: 4
good_list: 67 bad_list: 67
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
68商品目: https://page.auctions.yahoo.co.jp/jp/auction/f380790066 の取得開始
good: 1071 bad: 4
good_list: 68 bad_list: 68
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
69商品目: https://page.auctions.yahoo.co.jp/jp/auction/o350048469 の取得開始
good: 1071 bad: 4
good_list: 69 bad_list: 69
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
70商品目: https://page.auctions.yahoo.co.jp/jp/auction/427243009 の取得開始
good: 1071 bad: 4
good_list: 70 bad_list: 70
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
71商品目: https://page.auctions.yahoo.co.jp/jp/auction/e389635044 の取得開始
good: 1071 bad: 4
good_list

good: 35 bad: 1
good_list: 19 bad_list: 19
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
20商品目: https://page.auctions.yahoo.co.jp/jp/auction/b422144670 の取得開始
good: 228 bad: 0
good_list: 20 bad_list: 20
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
21商品目: https://page.auctions.yahoo.co.jp/jp/auction/r356185668 の取得開始
good: 35 bad: 1
good_list: 21 bad_list: 21
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
22商品目: https://page.auctions.yahoo.co.jp/jp/auction/c766142432 の取得開始
good: 115 bad: 0
good_list: 22 bad_list: 22
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
23商品目: https://page.auctions.yahoo.co.jp/jp/auction/r355084705 の取得開始
good: 3408 bad: 6
good_list: 23 bad_list: 23
画像枚数： 5
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
24商品目: https://page.auctions.yahoo.co.jp/jp/auction/s693387990 の取得開始
good: 3408 bad: 6
good_list: 24 bad_list: 24
画像枚数： 5
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
25商品目: https://page.auctions.yahoo.co.jp/jp/auction/f380645759 の取得開始
good: 129 bad: 0
good_list: 25 bad_list: 25
画像枚数： 3
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
26商品目: https://page

74商品目: https://page.auctions.yahoo.co.jp/jp/auction/v669558801 の取得開始
good: 2744 bad: 10
good_list: 74 bad_list: 74
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
75商品目: https://page.auctions.yahoo.co.jp/jp/auction/j596586713 の取得開始
good: 40540 bad: 190
good_list: 75 bad_list: 75
画像枚数： 3
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
76商品目: https://page.auctions.yahoo.co.jp/jp/auction/h428105233 の取得開始
good: 275 bad: 1
good_list: 76 bad_list: 76
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
77商品目: https://page.auctions.yahoo.co.jp/jp/auction/j598081247 の取得開始
good: 275 bad: 1
good_list: 77 bad_list: 77
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
78商品目: https://page.auctions.yahoo.co.jp/jp/auction/d394128479 の取得開始
good: 115 bad: 0
good_list: 78 bad_list: 78
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
79商品目: https://page.auctions.yahoo.co.jp/jp/auction/j597147297 の取得開始
good: 663 bad: 37
good_list: 79 bad_list: 79
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
80商品目: https://page.auctions.yahoo.co.jp/jp/auction/h425685878 の取得開始
good: 2744 bad: 10
good_

good: 115 bad: 0
good_list: 28 bad_list: 28
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
29商品目: https://page.auctions.yahoo.co.jp/jp/auction/f377525931 の取得開始
good: 115 bad: 0
good_list: 29 bad_list: 29
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
30商品目: https://page.auctions.yahoo.co.jp/jp/auction/d392759832 の取得開始
good: 115 bad: 0
good_list: 30 bad_list: 30
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
31商品目: https://page.auctions.yahoo.co.jp/jp/auction/j595873265 の取得開始
good: 275 bad: 1
good_list: 31 bad_list: 31
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
32商品目: https://page.auctions.yahoo.co.jp/jp/auction/j595138590 の取得開始
good: 663 bad: 37
good_list: 32 bad_list: 32
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
33商品目: https://page.auctions.yahoo.co.jp/jp/auction/h426129315 の取得開始
good: 275 bad: 1
good_list: 33 bad_list: 33
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
34商品目: https://page.auctions.yahoo.co.jp/jp/auction/e385128973 の取得開始
good: 15 bad: 0
good_list: 34 bad_list: 34
画像枚数： 3
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
35商品目: https://page

83商品目: https://page.auctions.yahoo.co.jp/jp/auction/t675407420 の取得開始
good: 232 bad: 0
good_list: 83 bad_list: 83
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
84商品目: https://page.auctions.yahoo.co.jp/jp/auction/v667369517 の取得開始
good: 2744 bad: 10
good_list: 84 bad_list: 84
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
85商品目: https://page.auctions.yahoo.co.jp/jp/auction/t675885276 の取得開始
good: 967 bad: 15
good_list: 85 bad_list: 85
画像枚数： 4
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
86商品目: https://page.auctions.yahoo.co.jp/jp/auction/u305755878 の取得開始
good: 449 bad: 0
good_list: 86 bad_list: 86
画像枚数： 3
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
87商品目: https://page.auctions.yahoo.co.jp/jp/auction/k414521332 の取得開始
good: 19 bad: 0
good_list: 87 bad_list: 87
画像枚数： 4
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
88商品目: https://page.auctions.yahoo.co.jp/jp/auction/x644918702 の取得開始
good: 683 bad: 4
good_list: 88 bad_list: 88
画像枚数： 2
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
89商品目: https://page.auctions.yahoo.co.jp/jp/auction/423689187 の取得開始
good: 115 bad: 0
good_list: 89

good: 967 bad: 2
good_list: 37 bad_list: 37
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
38商品目: https://page.auctions.yahoo.co.jp/jp/auction/422032729 の取得開始
good: 115 bad: 0
good_list: 38 bad_list: 38
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
39商品目: https://page.auctions.yahoo.co.jp/jp/auction/l548428561 の取得開始
good: 290 bad: 3
good_list: 39 bad_list: 39
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
40商品目: https://page.auctions.yahoo.co.jp/jp/auction/e384701236 の取得開始
good: 1321 bad: 51
good_list: 40 bad_list: 40
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
41商品目: https://page.auctions.yahoo.co.jp/jp/auction/e383281165 の取得開始
good: 1321 bad: 51
good_list: 41 bad_list: 41
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
42商品目: https://page.auctions.yahoo.co.jp/jp/auction/e380520626 の取得開始
good: 309 bad: 4
good_list: 42 bad_list: 42
画像枚数： 0
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
43商品目: https://page.auctions.yahoo.co.jp/jp/auction/l551399175 の取得開始
good: 120 bad: 1
good_list: 43 bad_list: 43
画像枚数： 4
取得完了
過剰な連続リクエスト回避のsleep:0.5秒
44商品目: https://p

In [14]:
df0=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品1.csv")
df1=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品101.csv")
df2=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品201.csv")
df3=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品301.csv")
df4=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品401.csv")
df5=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品501.csv")
df6=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品601.csv")
df7=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品701.csv")
df8=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品801.csv")
df9=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品901.csv")
df10=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品1001.csv")
df11=pd.read_csv("airpods+第2世代+新品/airpods+第2世代+新品1101.csv")

In [15]:
df_concat=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11],ignore_index=True,axis=0)

/Users/okazakisouichirou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
df_concat

,Unnamed: 0,bad,bidders,capture_count,end_price,end_time,good,is_free_shipping,is_prompt,product_name,setsumei,start_price,start_time,timedelta
0,0,0.0,1,送料無料,6380,2019-11-16 01:45:02,232.0,1,1,Apple 純正 AirPods 第2世代 新品 MV7N2J/A 充電ケース＋Lightn...,Apple AirPods with Charging Case MV7N2J/A 第２世代...,6380,2019-11-16 01:31:49,0.0
1,1,2.0,1,送料無料,16500,2019-11-15 16:33:55,176.0,1,1,[新品未使用品]★Apple AirPods 第2世代 with wireless Char...,【商品の説明】Apple AirPods 第2世代ブランド、メーカー：アップル型番：MRXJ...,16500,2019-11-15 15:27:01,1.0
2,2,0.0,2,送料無料,1880,2019-11-15 14:38:16,12.0,1,1,新品 セール AirPodsケース シリコン スターバックス カラビナ付き AirPods ...,ご覧いただきありがとうございます。 以下を必ず最後までお読み頂いた上で、入札お待ちしており...,1880,2019-11-08 11:43:09,170.0
3,3,10.0,1,未使用,6900,2019-11-15 13:43:36,2744.0,0,1,Apple純正 AirPods 第2世代 左 イヤホン MV7N2J/A 左耳のみ 新品未使用品,Apple純正 AirPods 第2世代 左 イヤホン MV7N2J/A 左耳のみ 新品未使...,6900,2019-11-14 05:38:46,32.0
4,4,0.0,1,送料無料,1880,2019-11-15 13:38:11,12.0,1,1,新品 セール AirPodsケース シリコン 恐竜 カラビナ付き AirPods 第1世代/...,ご覧いただきありがとうございます。 以下を必ず最後までお読み頂いた上で、入札お待ちしており...,1880,2019-11-08 11:43:08,169.0
5,5,0.0,1,未使用,19500,2019-11-15 12:44:34,241.0,0,1,【正規品】 新品 保証あり/AirPods (エアーポッズ/第2世代) with Charg...,ご覧いただきありがとうございます AirPods (エアーポッズ/第2世代) 【商品...,16980,2019-11-14 12:44:34,24.0
6,6,1.0,1,送料無料,7000,2019-11-15 05:30:58,232.0,1,1,【送料無料】 Apple 純正品 AirPods2 第2世代 MV7N2J/A 左のみ L 新品,2019年10月購入品です。納品書のコピー付きです。充電ケース・ケーブル・説明書・外箱は付属...,7000,2019-11-13 12:24:33,41.0
7,7,10.0,1,匿名配送,17500,2019-11-15 02:15:44,239.0,0,1,【新品未開封】 AirPods with Charging Case 第2世代 MV7N2J...,AirPods with Charging Case 新品未開封品 MV7N2J/A Mid...,17500,2019-11-13 20:28:34,29.0
8,8,3.0,1,（全1枚）,670,2019-11-15 00:54:53,86.0,0,1,新品 好評 エアポッド Airpodsケ-ス 1-6N 萌え萌え 人気 カバ- 全面保護ケ-...,▼商品詳細▼Airpodsケ-ス エアポッド カバ- 全面保護ケ-ス 耐衝撃 紛失防止 Ai...,570,2019-11-14 12:12:56,12.0
9,9,3.0,1,送料無料,18500,2019-11-14 23:52:02,263.0,1,1,【定額＆送料無料】新品未開封：Apple(アップル)AirPods with Wireles...,赤レタパ送料無料,18500,2019-11-13 06:16:23,41.0


In [17]:
df_concat.to_csv("CSV/ヤフオク詳細11_16.csv")